# 下面是用CLIP模型做的推理

In [ ]:
from transformers import CLIPProcessor, CLIPModel
from PIL import Image


# 加载预训练的 CLIP 模型和处理器
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
print(model)
print('-'*100)
print(processor)
# 加载图像
image = Image.open('luke.jpg')

# 定义文本描述
texts = ["a photo of a cat", "a photo of a dog", "a photo of a girl"]

# 处理输入
inputs = processor(text=texts, images=image, return_tensors="pt", padding=True)
print(inputs)
# 前向传播
outputs = model(**inputs)

logits_per_image = outputs.logits_per_image  # 图像与文本的相似度
probs = logits_per_image.softmax(dim=1)  # 转换为概率

# 输出结果
print("Probabilities:", probs)

CLIPModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 512)
      (position_embedding): Embedding(77, 512)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=512, out_features=2048, bias=True)
            (fc2): Linear(in_features=2048, out_features=512, bias=True)
          )
          (layer_norm2): LayerNorm((512,), eps=1e-05,

In [4]:
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import torch

# 加载预训练的 CLIP 模型和处理器
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

print("CLIP Model Architecture:")
print(model)
print('-'*100)
# print("CLIP Processor:")
# print(processor)
print('='*100)

# 加载图像
image = Image.open('luke.jpg')

# 定义文本描述
texts = ["a photo of a cat", "a photo of a dog", "a photo of a girl"]

# 处理输入
inputs = processor(text=texts, images=image, return_tensors="pt", padding=True)
print("Processed Inputs:")
print(f"Input keys: {list(inputs.keys())}")
for key, value in inputs.items():
    if isinstance(value, torch.Tensor):
        print(f"  {key}: {value.shape}")
    else:
        print(f"  {key}: {type(value)}")
print('-'*50)

# 设置模型为评估模式
model.eval()

# 使用torch.no_grad()进行推理以节省内存
with torch.no_grad():
    print("Forward Pass Analysis:")
    print('-'*50)

    # 分别处理文本和图像以获得中间结果

    # 1. 文本编码
    print("1. Text Encoding:")
    text_inputs = {
        'input_ids': inputs['input_ids'],
        'attention_mask': inputs['attention_mask']
    }
    print(f"   Text input_ids shape: {text_inputs['input_ids'].shape}")
    print(f"   Text attention_mask shape: {text_inputs['attention_mask'].shape}")

    # 通过text_model获取文本特征
    text_outputs = model.text_model(**text_inputs)
    text_embeds = model.text_projection(text_outputs.pooler_output)

    print(f"   Text model hidden states shape: {text_outputs.last_hidden_state.shape}")
    print(f"   Text model pooler output shape: {text_outputs.pooler_output.shape}")
    print(f"   Text embeddings (after projection) shape: {text_embeds.shape}")
    print()

    # 2. 图像编码
    print("2. Vision Encoding:")
    vision_inputs = {
        'pixel_values': inputs['pixel_values']
    }
    print(f"   Vision pixel_values shape: {vision_inputs['pixel_values'].shape}")

    # 通过vision_model获取图像特征
    vision_outputs = model.vision_model(**vision_inputs)
    image_embeds = model.visual_projection(vision_outputs.pooler_output)

    print(f"   Vision model hidden states shape: {vision_outputs.last_hidden_state.shape}")
    print(f"   Vision model pooler output shape: {vision_outputs.pooler_output.shape}")
    print(f"   Image embeddings (after projection) shape: {image_embeds.shape}")
    print()

    # 3. 完整前向传播
    print("3. Complete Forward Pass:")
    outputs = model(**inputs)

    print(f"   Final text features shape: {outputs.text_embeds.shape}")
    print(f"   Final image features shape: {outputs.image_embeds.shape}")
    print(f"   Logits per image shape: {outputs.logits_per_image.shape}")
    print(f"   Logits per text shape: {outputs.logits_per_text.shape}")
    print()

    # 4. 相似度计算详解
    print("4. Similarity Computation Details:")
    logits_per_image = outputs.logits_per_image
    logits_per_text = outputs.logits_per_text

    print(f"   Image-to-text logits: {logits_per_image.shape}")
    print(f"   Text-to-image logits: {logits_per_text.shape}")
    print(f"   Logit scale parameter: {model.logit_scale.exp().item():.4f}")

    # 转换为概率
    probs_image_to_text = logits_per_image.softmax(dim=1)
    probs_text_to_image = logits_per_text.softmax(dim=0)

    print(f"   Image-to-text probabilities shape: {probs_image_to_text.shape}")
    print(f"   Text-to-image probabilities shape: {probs_text_to_image.shape}")
    print()

    # 5. 详细结果展示
    print("5. Detailed Results:")
    print('-'*30)

    # 原始相似度分数
    print("Raw similarity scores (logits):")
    for i, text in enumerate(texts):
        print(f"   '{text}': {logits_per_image[0, i].item():.4f}")
    print()

    # 概率分布
    print("Probability distribution:")
    probs = probs_image_to_text[0]  # 取第一张（也是唯一一张）图像的概率
    for i, text in enumerate(texts):
        print(f"   '{text}': {probs[i].item():.4f} ({probs[i].item()*100:.2f}%)")
    print()

    # 最匹配的文本
    best_match_idx = torch.argmax(probs).item()
    print(f"Best match: '{texts[best_match_idx]}' with probability {probs[best_match_idx].item():.4f}")
    print()

# 6. 模型参数统计
print("6. Model Parameter Statistics:")
print('-'*40)
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

# 各组件参数统计
text_params = sum(p.numel() for p in model.text_model.parameters())
vision_params = sum(p.numel() for p in model.vision_model.parameters())
text_proj_params = sum(p.numel() for p in model.text_projection.parameters())
visual_proj_params = sum(p.numel() for p in model.visual_projection.parameters())

print(f"Text model parameters: {text_params:,}")
print(f"Vision model parameters: {vision_params:,}")
print(f"Text projection parameters: {text_proj_params:,}")
print(f"Visual projection parameters: {visual_proj_params:,}")
print(f"Logit scale parameter: 1")

print('='*100)
print("Analysis Complete!")

CLIP Model Architecture:
CLIPModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 512)
      (position_embedding): Embedding(77, 512)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=512, out_features=2048, bias=True)
            (fc2): Linear(in_features=2048, out_features=512, bias=True)
          )
          (layer_norm2): Lay

# 下面是理解CLIP写的简单样例

相似度是在batch内计算，论文中的N就是batch size

In [ ]:
import torch  # 导入PyTorch库
import torch.nn as nn  # 导入PyTorch神经网络模块
import torch.nn.functional as F  # 导入PyTorch函数式API

# 定义简单的图像编码器和文本编码器
class ImageEncoder(nn.Module):  # 定义图像编码器类，继承自nn.Module
    def __init__(self, embed_dim):  # 初始化方法，接收嵌入维度参数
        super(ImageEncoder, self).__init__()  # 调用父类初始化方法
        self.fc = nn.Linear(1024, embed_dim)  # 假设输入图像特征维度为 1024，创建线性层将其映射到嵌入维度

    def forward(self, x):  # 前向传播方法
        return self.fc(x)  # 返回线性变换后的结果

class TextEncoder(nn.Module):  # 定义文本编码器类，继承自nn.Module
    def __init__(self, embed_dim):  # 初始化方法，接收嵌入维度参数
        super(TextEncoder, self).__init__()  # 调用父类初始化方法
        self.fc = nn.Linear(512, embed_dim)  # 假设输入文本特征维度为 512，创建线性层将其映射到嵌入维度

    def forward(self, x):  # 前向传播方法
        return self.fc(x)  # 返回线性变换后的结果

# 定义 CLIP 模型
class CLIP(nn.Module):  # 定义CLIP模型类，继承自nn.Module
    def __init__(self, embed_dim):  # 初始化方法，接收嵌入维度参数
        super(CLIP, self).__init__()  # 调用父类初始化方法
        self.image_encoder = ImageEncoder(embed_dim)  # 创建图像编码器实例
        self.text_encoder = TextEncoder(embed_dim)  # 创建文本编码器实例
        self.logit_scale = nn.Parameter(torch.ones([]) * 2.6592)  # 可学习的温度参数，初始化为2.6592

    def forward(self, images, texts):  # 前向传播方法，接收图像和文本输入
        # 编码图像和文本
        image_features = self.image_encoder(images)  # 使用图像编码器处理图像
        text_features = self.text_encoder(texts)  # 使用文本编码器处理文本

        # 归一化特征
        image_features = F.normalize(image_features, dim=-1)  # 对图像特征进行L2归一化
        text_features = F.normalize(text_features, dim=-1)  # 对文本特征进行L2归一化

        # 计算相似度矩阵
        logit_scale = self.logit_scale.exp()  # 计算温度参数的指数
        logits_per_image = logit_scale * image_features @ text_features.t()  # 计算图像-文本相似度矩阵,@表示矩阵乘法
        logits_per_text = logits_per_image.t()  # 计算文本-图像相似度矩阵（转置）

        return logits_per_image, logits_per_text  # 返回两个相似度矩阵

# 定义对比损失函数
def clip_loss(logits_per_image, logits_per_text):  # 定义CLIP损失函数，接收两个相似度矩阵
    # 计算图像到文本的损失
    #labels的值是(0,1,2,3),因为logits_per_image的shape是[4,4]，所以labels的shape也是[4]
    labels = torch.arange(logits_per_image.size(0)).to(logits_per_image.device)  # 创建标签，表示对角线位置为正样本
    loss_image = F.cross_entropy(logits_per_image, labels)  # 计算图像到文本的交叉熵损失

    # 计算文本到图像的损失
    loss_text = F.cross_entropy(logits_per_text, labels)  # 计算文本到图像的交叉熵损失

    # 总损失
    total_loss = (loss_image + loss_text) / 2  # 计算总损失为两个方向损失的平均值
    return total_loss  # 返回总损失

# 示例数据
batch_size = 4  # 设置批次大小为4
embed_dim = 64  # 设置嵌入维度为64
images = torch.randn(batch_size, 1024)  # 生成随机图像特征，维度为[4, 1024]
texts = torch.randn(batch_size, 512)    # 生成随机文本特征，维度为[4, 512]

# 初始化模型
model = CLIP(embed_dim)  # 创建CLIP模型实例，嵌入维度为64

# 前向传播
logits_per_image, logits_per_text = model(images, texts)  # 执行模型前向传播，获取相似度矩阵
print("logits_per_image:", logits_per_image.shape)  # 打印图像-文本相似度矩阵的形状
print("logits_per_text:", logits_per_text.shape)  # 打印文本-图像相似度矩阵的形状
# 计算损失
loss = clip_loss(logits_per_image, logits_per_text)  # 计算对比损失
print("Loss:", loss.item())  # 打印损失值

logits_per_image: torch.Size([4, 4])
logits_per_text: torch.Size([4, 4])
Loss: 3.2554702758789062


In [ ]:
import math  # 导入数学库，用于数学计算如平方根等
import torch  # 导入PyTorch库，用于深度学习模型构建和训练
import torch.nn as nn  # 导入PyTorch神经网络模块，提供构建神经网络的各种组件
import torch.nn.functional as F  # 导入PyTorch函数式API，提供各种激活函数和损失函数等

class PatchEmbedding(nn.Module):  # 定义图像分块嵌入类，将图像分成小块并嵌入到向量空间
    def __init__(self, image_size=224, patch_size=16, in_channels=3, embed_dim=768):  # 初始化方法，设置默认参数
        super().__init__()  # 调用父类初始化方法
        self.image_size = image_size  # 存储图像大小，默认为224x224像素
        self.patch_size = patch_size  # 存储patch的大小，用于将图像分割成固定大小的patch，patch_size通常为16，表示将图像分割成16x16大小的小块
        self.num_patches = (image_size // patch_size) ** 2  # 计算patch的总数量，等于(图像大小/patch大小)的平方

        # 将图像分成patch并进行线性投影，使用卷积层实现，卷积核大小和步长都等于patch_size
        self.projection = nn.Conv2d(in_channels, embed_dim, kernel_size=patch_size, stride=patch_size)

        # 位置编码相关参数
        self.cls_token = nn.Parameter(torch.randn(1, 1, embed_dim))  # 创建可学习的分类token，用于整体图像表示
        self.pos_embed = nn.Parameter(torch.randn(1, self.num_patches + 1, embed_dim))  # 创建可学习的位置编码，+1是为了包含cls_token的位置,看下面forward即可理解

    def forward(self, x):  # 前向传播方法
        B = x.shape[0]  # 获取批次大小
        # 将图像转换为patch序列：先通过卷积层，然后将空间维度展平，最后调整维度顺序
        x = self.projection(x).flatten(2).transpose(1, 2)

        # 添加分类token：将cls_token扩展到批次维度，然后与patch序列拼接
        cls_tokens = self.cls_token.expand(B, -1, -1)  # 扩展cls_token到批次大小
        x = torch.cat((cls_tokens, x), dim=1)  # 在序列维度上拼接cls_token和patch序列

        # 添加位置编码：将位置编码加到token序列上
        x = x + self.pos_embed  # 添加位置编码，提供序列中位置信息
        return x  # 返回处理后的序列

class MultiHeadAttention(nn.Module):  # 定义多头注意力机制类
    def __init__(self, embed_dim, num_heads):  # 初始化方法，接收嵌入维度和注意力头数
        super().__init__()  # 调用父类初始化方法
        self.num_heads = num_heads  # 存储注意力头数量
        self.head_dim = embed_dim // num_heads  # 计算每个注意力头的维度
        self.scale = self.head_dim ** -0.5  # 计算缩放因子，用于缩放注意力分数

        self.qkv = nn.Linear(embed_dim, embed_dim * 3)  # 创建线性层，用于同时生成查询(Q)、键(K)和值(V)
        self.proj = nn.Linear(embed_dim, embed_dim)  # 创建输出投影层，将多头注意力的结果映射回原始维度

    def forward(self, x):  # 前向传播方法
        B, N, C = x.shape  # 获取输入的批次大小、序列长度和特征维度
        # 生成查询、键、值并重塑维度以适应多头注意力计算
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, self.head_dim).permute(2, 0, 3, 1, 4)  # 重塑维度
        q, k, v = qkv[0], qkv[1], qkv[2]  # 分离查询、键、值

        # 计算注意力分数：查询和键的矩阵乘法，然后应用缩放
        attn = (q @ k.transpose(-2, -1)) * self.scale  # 计算缩放点积注意力
        attn = attn.softmax(dim=-1)  # 对注意力分数应用softmax归一化

        # 应用注意力权重到值上，并重塑回原始维度
        x = (attn @ v).transpose(1, 2).reshape(B, N, C)  # 计算加权和并重塑维度
        x = self.proj(x)  # 通过输出投影层
        return x  # 返回注意力的输出

class ImageEncoder(nn.Module):  # 定义图像编码器类
    def __init__(self, embed_dim, image_size=224, patch_size=16, num_layers=12, num_heads=12, mlp_ratio=4.0):  # 初始化方法
        super().__init__()  # 调用父类初始化方法
        # 创建图像分块嵌入层
        self.patch_embed = PatchEmbedding(image_size, patch_size, 3, embed_dim)  # 创建图像分块嵌入实例

        # Transformer编码器层：创建多个Transformer层
        self.layers = nn.ModuleList([])  # 创建模块列表，用于存储Transformer层
        for _ in range(num_layers):  # 循环创建指定数量的Transformer层
            layer = nn.Sequential(  # 使用Sequential容器组织层的顺序
                nn.LayerNorm(embed_dim),  # 第一个层归一化
                MultiHeadAttention(embed_dim, num_heads),  # 多头注意力层
                nn.LayerNorm(embed_dim),  # 第二个层归一化
                nn.Sequential(  # MLP块，包含两个线性层和GELU激活函数
                    nn.Linear(embed_dim, int(embed_dim * mlp_ratio)),  # 第一个线性层，扩展维度
                    nn.GELU(),  # GELU激活函数
                    nn.Linear(int(embed_dim * mlp_ratio), embed_dim)  # 第二个线性层，恢复维度
                )
            )
            self.layers.append(layer)  # 将创建的层添加到模块列表中

        self.norm = nn.LayerNorm(embed_dim)  # 最终的层归一化

    def forward(self, x):  # 前向传播方法
        x = self.patch_embed(x)  # 通过图像分块嵌入层处理输入

        for layer in self.layers:  # 遍历所有Transformer层
            x = x + layer[0](x)  # 第一个残差连接：原始输入 + 层归一化和注意力的输出
            x = x + layer[2](layer[3](layer[1](x)))  # 第二个残差连接：中间结果 + 层归一化和MLP的输出

        x = self.norm(x)  # 应用最终的层归一化
        return x[:, 0]  # 只返回[CLS]token的特征，用作整个图像的表示

class TextEncoder(nn.Module):  # 定义文本编码器类，继承自nn.Module
    def __init__(self, embed_dim):  # 初始化方法，接收嵌入维度参数
        super(TextEncoder, self).__init__()  # 调用父类初始化方法
        self.fc = nn.Linear(512, embed_dim)  # 假设输入文本特征维度为 512，创建线性层将其映射到嵌入维度

    def forward(self, x):  # 前向传播方法
        # 实现类似CLIP模型中的注意力机制
        batch_size, seq_len = x.shape[0], x.shape[1]  # 获取批次大小和序列长度

        # 假设我们有一个简单的自注意力机制
        # 创建QKV投影
        hidden_size = 512  # 设置隐藏层大小
        q_proj = nn.Linear(seq_len, hidden_size).to(x.device)  # 创建查询投影层，并移动到与输入相同的设备上
        k_proj = nn.Linear(seq_len, hidden_size).to(x.device)  # 创建键投影层
        v_proj = nn.Linear(seq_len, hidden_size).to(x.device)  # 创建值投影层
        out_proj = nn.Linear(hidden_size, hidden_size).to(x.device)  # 创建输出投影层

        # 计算QKV
        q = q_proj(x)  # 生成查询向量
        k = k_proj(x)  # 生成键向量
        v = v_proj(x)  # 生成值向量

        # 计算注意力分数
        attention_scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(hidden_size)  # 计算缩放点积注意力分数
        attention_probs = F.softmax(attention_scores, dim=-1)  # 对注意力分数应用softmax归一化

        # 应用注意力
        context_layer = torch.matmul(attention_probs, v)  # 计算注意力加权和

        # 输出投影
        attention_output = out_proj(context_layer)  # 通过输出投影层

        # 最后通过线性层
        return self.fc(attention_output + x)  # 添加残差连接并返回结果

# 定义 CLIP 模型
class CLIP(nn.Module):  # 定义CLIP模型类，继承自nn.Module
    def __init__(self, embed_dim):  # 初始化方法，接收嵌入维度参数
        super(CLIP, self).__init__()  # 调用父类初始化方法
        self.image_encoder = ImageEncoder(embed_dim)  # 创建图像编码器实例
        self.text_encoder = TextEncoder(embed_dim)  # 创建文本编码器实例
        self.logit_scale = nn.Parameter(torch.ones([]) * 2.6592)  # 可学习的温度参数，初始化为2.6592

    def forward(self, images, texts):  # 前向传播方法，接收图像和文本输入
        # 编码图像和文本
        image_features = self.image_encoder(images)  # 使用图像编码器处理图像
        text_features = self.text_encoder(texts)  # 使用文本编码器处理文本

        # 归一化特征
        image_features = F.normalize(image_features, dim=-1)  # 对图像特征进行L2归一化
        text_features = F.normalize(text_features, dim=-1)  # 对文本特征进行L2归一化

        # 计算相似度矩阵
        logit_scale = self.logit_scale.exp()  # 计算温度参数的指数
        logits_per_image = logit_scale * image_features @ text_features.t()  # 计算图像-文本相似度矩阵
        logits_per_text = logits_per_image.t()  # 计算文本-图像相似度矩阵（转置）

        return logits_per_image, logits_per_text  # 返回两个相似度矩阵

# 定义对比损失函数
def clip_loss(logits_per_image, logits_per_text):  # 定义CLIP损失函数，接收两个相似度矩阵
    # 计算图像到文本的损失
    labels = torch.arange(logits_per_image.size(0)).to(logits_per_image.device)  # 创建标签，表示对角线位置为正样本
    loss_image = F.cross_entropy(logits_per_image, labels)  # 计算图像到文本的交叉熵损失

    # 计算文本到图像的损失
    loss_text = F.cross_entropy(logits_per_text, labels)  # 计算文本到图像的交叉熵损失

    # 总损失
    total_loss = (loss_image + loss_text) / 2  # 计算总损失为两个方向损失的平均值
    return total_loss  # 返回总损失

# 示例数据
batch_size = 4  # 设置批次大小为4
embed_dim = 768  # 设置嵌入维度为768
images = torch.randn(batch_size, 3, 224, 224)  # 生成随机图像数据，维度为[4, 3, 224, 224]
texts = torch.randn(batch_size, 512)    # 生成随机文本特征，维度为[4, 512]

# 初始化模型
model = CLIP(embed_dim)  # 创建CLIP模型实例，嵌入维度为768

# 前向传播
logits_per_image, logits_per_text = model(images, texts)  # 执行模型前向传播，获取相似度矩阵
print("logits_per_image:", logits_per_image.shape)  # 打印图像-文本相似度矩阵的形状
print("logits_per_text:", logits_per_text.shape)  # 打印文本-图像相似度矩阵的形状
# 计算损失
loss = clip_loss(logits_per_image, logits_per_text)  # 计算对比损失
print("Loss:", loss.item())  # 打印损失值

logits_per_image: torch.Size([4, 4])
logits_per_text: torch.Size([4, 4])
Loss: 1.563080072402954
